In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message("Hi I'm Sangmoo, I live in South Korea", "Wow that is so cold!")

In [2]:
memory.load_memory_variables({"input": "who is Sangmoo"})

{'history': [HumanMessage(content="Hi I'm Sangmoo, I live in South Korea"),
  AIMessage(content='Wow that is so cold!')]}

In [3]:
add_message("Sangmoo likes beef", "Wow that is so cold!")

In [4]:
memory.load_memory_variables({"inputs": "what does Sangmoo like"})

{'history': [HumanMessage(content="Hi I'm Sangmoo, I live in South Korea"),
  AIMessage(content='Wow that is so cold!'),
  HumanMessage(content='Sangmoo likes beef'),
  AIMessage(content='Wow that is so cold!')]}